# Classification Methods

Implementation of various classification algorithms including Logistic Regression, Linear Discriminant Analysis (LDA), Quadratic Discriminant Analysis (QDA), and K-Nearest Neighbors (KNN).

## Contents
1. Data Loading and Exploration
2. Logistic Regression
3. Model Evaluation and Confusion Matrix
4. Cross-Validation
5. Linear Discriminant Analysis (LDA)
6. Quadratic Discriminant Analysis (QDA)
7. K-Nearest Neighbors (KNN)
8. Feature Scaling

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Load Smarket Dataset

Stock market data with daily percentage returns for the S&P 500 over a 5-year period (2001-2005).

In [ ]:
# Load Smarket dataset
url = "https://raw.githubusercontent.com/selva86/datasets/master/Smarket.csv"
Smarket = pd.read_csv(url)

print(f"Dataset shape: {Smarket.shape}")
print(f"\nColumn names:")
print(Smarket.columns.tolist())
Smarket.head()

In [ ]:
# Dataset summary
print("Summary Statistics:")
Smarket.describe()

In [ ]:
# Check data types and missing values
print("Data Info:")
print(Smarket.info())
print(f"\nTarget variable distribution:")
print(Smarket['Direction'].value_counts())

### Exploratory Data Analysis

In [ ]:
# Correlation matrix (exclude Direction which is categorical)
numeric_cols = Smarket.select_dtypes(include=[np.number]).columns
correlation = Smarket[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

print("\nKey observation: Low correlation between Today and Lag1-5")
print("This suggests past returns have little predictive power for future returns")

In [ ]:
# Volume trend over time
plt.figure(figsize=(12, 6))
plt.plot(Smarket['Volume'], alpha=0.7)
plt.xlabel('Day')
plt.ylabel('Volume')
plt.title('Trading Volume Over Time (Increasing Trend)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Logistic Regression

Binary classification: Predict market direction (Up/Down) based on previous days' returns.

In [ ]:
# Prepare features and target
X = Smarket[['Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume']]
y = (Smarket['Direction'] == 'Up').astype(int)  # Up=1, Down=0

# Fit logistic regression
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X, y)

print("Logistic Regression Coefficients:")
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': log_reg.coef_[0]
}).sort_values('Coefficient', ascending=False)
print(coef_df)
print(f"\nIntercept: {log_reg.intercept_[0]:.4f}")

In [ ]:
# Predictions and probabilities
y_pred = log_reg.predict(X)
y_probs = log_reg.predict_proba(X)[:, 1]  # Probability of Up

print("First 10 predictions:")
results_df = pd.DataFrame({
    'Actual': y[:10].values,
    'Predicted': y_pred[:10],
    'Prob(Up)': y_probs[:10]
})
print(results_df)

## Confusion Matrix and Accuracy

In [ ]:
# Confusion matrix
cm = confusion_matrix(y, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Training Data')
plt.tight_layout()
plt.show()

# Calculate metrics
accuracy = accuracy_score(y, y_pred)
print(f"\nTraining Accuracy: {accuracy:.4f}")
print(f"Training Error Rate: {1 - accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y, y_pred, target_names=['Down', 'Up']))

## Cross-Validation: Train/Test Split

Train on years 2001-2004, test on year 2005.

In [ ]:
# Create train/test split based on year
train_mask = Smarket['Year'] < 2005
test_mask = Smarket['Year'] >= 2005

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"\nTest set year distribution:")
print(Smarket[test_mask]['Year'].value_counts().sort_index())

In [ ]:
# Train on training data
log_reg_cv = LogisticRegression(max_iter=1000, random_state=42)
log_reg_cv.fit(X_train, y_train)

# Predict on test data
y_pred_test = log_reg_cv.predict(X_test)
y_probs_test = log_reg_cv.predict_proba(X_test)[:, 1]

# Confusion matrix for test data
cm_test = confusion_matrix(y_test, y_pred_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Test Data (2005)')
plt.tight_layout()
plt.show()

# Test accuracy
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print(f"Test Error Rate: {1 - test_accuracy:.4f}")

## Feature Selection: Using Only Lag1 and Lag2

Remove predictors with high p-values to reduce variance.

In [ ]:
# Use only Lag1 and Lag2 (lowest p-values from full model)
X_reduced = Smarket[['Lag1', 'Lag2']]
X_train_red = X_reduced[train_mask]
X_test_red = X_reduced[test_mask]

# Fit model
log_reg_red = LogisticRegression(max_iter=1000, random_state=42)
log_reg_red.fit(X_train_red, y_train)

# Predict
y_pred_red = log_reg_red.predict(X_test_red)

# Confusion matrix
cm_red = confusion_matrix(y_test, y_pred_red)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_red, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Reduced Model (Lag1 + Lag2)')
plt.tight_layout()
plt.show()

test_accuracy_red = accuracy_score(y_test, y_pred_red)
print(f"\nTest Accuracy (Reduced Model): {test_accuracy_red:.4f}")
print(f"Improvement: {test_accuracy_red - test_accuracy:.4f}")

In [ ]:
# Predict for new data
new_data = pd.DataFrame({
    'Lag1': [1.2, 1.5],
    'Lag2': [1.1, -0.8]
})

predictions = log_reg_red.predict(new_data)
probabilities = log_reg_red.predict_proba(new_data)[:, 1]

print("Predictions for new data:")
results = pd.DataFrame({
    'Lag1': new_data['Lag1'],
    'Lag2': new_data['Lag2'],
    'Prediction': ['Up' if p == 1 else 'Down' for p in predictions],
    'Prob(Up)': probabilities
})
print(results)

## Linear Discriminant Analysis (LDA)

Alternative to logistic regression for binary and multi-class classification.

In [ ]:
# Fit LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_red, y_train)

# Predictions
y_pred_lda = lda.predict(X_test_red)
y_probs_lda = lda.predict_proba(X_test_red)

print("LDA Model Parameters:")
print(f"Priors: {lda.priors_}")
print(f"\nClass means:")
print(pd.DataFrame(lda.means_, columns=['Lag1', 'Lag2'], index=['Down', 'Up']))

In [ ]:
# Confusion matrix for LDA
cm_lda = confusion_matrix(y_test, y_pred_lda)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_lda, annot=True, fmt='d', cmap='Purples',
            xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - LDA')
plt.tight_layout()
plt.show()

lda_accuracy = accuracy_score(y_test, y_pred_lda)
print(f"\nLDA Test Accuracy: {lda_accuracy:.4f}")

In [ ]:
# Posterior probabilities
print("First 20 posterior probabilities (Down class):")
print(y_probs_lda[:20, 0])
print("\nFirst 20 predictions:")
print(['Up' if p == 1 else 'Down' for p in y_pred_lda[:20]])

## Quadratic Discriminant Analysis (QDA)

Allows for non-linear decision boundaries between classes.

In [ ]:
# Fit QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_red, y_train)

# Predictions
y_pred_qda = qda.predict(X_test_red)

# Confusion matrix
cm_qda = confusion_matrix(y_test, y_pred_qda)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_qda, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - QDA')
plt.tight_layout()
plt.show()

qda_accuracy = accuracy_score(y_test, y_pred_qda)
print(f"\nQDA Test Accuracy: {qda_accuracy:.4f}")
print(f"Improvement over LDA: {qda_accuracy - lda_accuracy:.4f}")

## K-Nearest Neighbors (KNN)

In [ ]:
# KNN with K=1
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train_red, y_train)
y_pred_knn1 = knn1.predict(X_test_red)

cm_knn1 = confusion_matrix(y_test, y_pred_knn1)
knn1_accuracy = accuracy_score(y_test, y_pred_knn1)

print(f"KNN (K=1) Test Accuracy: {knn1_accuracy:.4f}")
print("\nConfusion Matrix:")
print(cm_knn1)

In [ ]:
# KNN with K=3
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(X_train_red, y_train)
y_pred_knn3 = knn3.predict(X_test_red)

cm_knn3 = confusion_matrix(y_test, y_pred_knn3)
knn3_accuracy = accuracy_score(y_test, y_pred_knn3)

print(f"KNN (K=3) Test Accuracy: {knn3_accuracy:.4f}")
print("\nConfusion Matrix:")
print(cm_knn3)

## Model Comparison

In [ ]:
# Compare all models
results_comparison = pd.DataFrame({
    'Model': ['Logistic Regression (All)', 'Logistic Regression (Lag1+Lag2)', 
              'LDA', 'QDA', 'KNN (K=1)', 'KNN (K=3)'],
    'Test Accuracy': [
        test_accuracy,
        test_accuracy_red,
        lda_accuracy,
        qda_accuracy,
        knn1_accuracy,
        knn3_accuracy
    ]
}).sort_values('Test Accuracy', ascending=False)

print("\n" + "="*50)
print("MODEL COMPARISON")
print("="*50)
print(results_comparison.to_string(index=False))
print("="*50)

In [ ]:
# Visualize model comparison
plt.figure(figsize=(10, 6))
plt.barh(results_comparison['Model'], results_comparison['Test Accuracy'], color='skyblue')
plt.xlabel('Test Accuracy')
plt.title('Model Performance Comparison')
plt.xlim([0.4, 0.65])
for i, v in enumerate(results_comparison['Test Accuracy']):
    plt.text(v + 0.005, i, f'{v:.3f}', va='center')
plt.tight_layout()
plt.show()

## Caravan Insurance Dataset

Predicting whether customers will purchase caravan insurance.

In [ ]:
# Load Caravan dataset
try:
    url_caravan = "https://raw.githubusercontent.com/selva86/datasets/master/Caravan.csv"
    Caravan = pd.read_csv(url_caravan)
    print(f"Dataset shape: {Caravan.shape}")
    
    # Check target variable
    print(f"\nPurchase distribution:")
    print(Caravan['Purchase'].value_counts())
    print(f"\nPurchase rate: {Caravan['Purchase'].value_counts(normalize=True)['Yes']:.1%}")
    
except:
    print("Could not load Caravan dataset. Skipping this section.")

### Feature Scaling

Standardize features to have mean=0 and variance=1. Critical for KNN!

In [ ]:
if 'Caravan' in locals():
    # Separate features and target
    X_caravan = Caravan.drop('Purchase', axis=1)
    y_caravan = (Caravan['Purchase'] == 'Yes').astype(int)
    
    # Check variance before scaling
    print("Before scaling:")
    print(f"Variance of first feature: {X_caravan.iloc[:, 0].var():.4f}")
    print(f"Variance of second feature: {X_caravan.iloc[:, 1].var():.4f}")
    print(f"Mean of first feature: {X_caravan.iloc[:, 0].mean():.4f}")
    
    # Standardize
    scaler = StandardScaler()
    X_caravan_scaled = scaler.fit_transform(X_caravan)
    X_caravan_scaled = pd.DataFrame(X_caravan_scaled, columns=X_caravan.columns)
    
    print("\nAfter scaling:")
    print(f"Variance of first feature: {X_caravan_scaled.iloc[:, 0].var():.4f}")
    print(f"Variance of second feature: {X_caravan_scaled.iloc[:, 1].var():.4f}")
    print(f"Mean of first feature: {X_caravan_scaled.iloc[:, 0].mean():.10f}")

In [ ]:
if 'Caravan' in locals():
    # Train/test split: first 1000 as test
    X_train_car = X_caravan_scaled.iloc[1000:]
    X_test_car = X_caravan_scaled.iloc[:1000]
    y_train_car = y_caravan.iloc[1000:]
    y_test_car = y_caravan.iloc[:1000]
    
    print(f"Training set size: {len(X_train_car)}")
    print(f"Test set size: {len(X_test_car)}")
    print(f"\nBaseline (predict No for all): {(y_test_car == 0).mean():.1%}")

### KNN on Caravan Data

In [ ]:
if 'Caravan' in locals():
    # KNN with different K values
    k_values = [1, 2, 3, 5]
    
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train_car, y_train_car)
        y_pred = knn.predict(X_test_car)
        
        cm = confusion_matrix(y_test_car, y_pred)
        error_rate = 1 - accuracy_score(y_test_car, y_pred)
        
        # True Positive Rate (for those who buy)
        if cm[1, 1] + cm[1, 0] > 0:
            tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0])
        else:
            tpr = 0
        
        print(f"\n{'='*50}")
        print(f"KNN with K={k}")
        print(f"{'='*50}")
        print(f"Error Rate: {error_rate:.4f}")
        print(f"True Positive Rate: {tpr:.1%}")
        print(f"\nConfusion Matrix:")
        print(cm)

### Logistic Regression on Caravan with Different Cutoffs

In [ ]:
if 'Caravan' in locals():
    # Fit logistic regression
    log_reg_car = LogisticRegression(max_iter=1000, random_state=42)
    log_reg_car.fit(X_train_car, y_train_car)
    
    # Get probabilities
    y_probs_car = log_reg_car.predict_proba(X_test_car)[:, 1]
    
    # Test different cutoffs
    cutoffs = [0.5, 0.25]
    
    for cutoff in cutoffs:
        y_pred_car = (y_probs_car > cutoff).astype(int)
        cm = confusion_matrix(y_test_car, y_pred_car)
        
        # True Positive Rate
        if cm[1, 1] + cm[1, 0] > 0:
            tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0])
        else:
            tpr = 0
        
        print(f"\n{'='*50}")
        print(f"Logistic Regression with cutoff = {cutoff}")
        print(f"{'='*50}")
        print(f"True Positive Rate: {tpr:.1%}")
        print(f"\nConfusion Matrix:")
        print(cm)
        print(f"\nInterpretation: {tpr:.1%} of actual buyers were correctly identified")

## Summary

This notebook covered:
- **Logistic Regression**: Binary classification with probability outputs
- **LDA**: Assumes Gaussian distributions with common covariance
- **QDA**: Allows different covariances for each class (non-linear boundaries)
- **KNN**: Non-parametric method based on nearest neighbors
- **Feature Scaling**: Critical for distance-based methods like KNN
- **Confusion Matrix**: Evaluating classification performance
- **Threshold Tuning**: Adjusting decision boundaries for imbalanced data

**Key Takeaways:**
- No single model is always best - depends on data
- Feature scaling is crucial for KNN and other distance-based methods
- For imbalanced data, accuracy alone is misleading - look at precision/recall
- Lower cutoffs can improve sensitivity at cost of specificity